In [1]:
pip install scrapy psycopg2-binary

  Using cached Scrapy-2.6.1-py2.py3-none-any.whl (264 kB)
  Using cached psycopg2_binary-2.9.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Using cached Protego-0.2.1-py2.py3-none-any.whl (8.2 kB)
  Using cached parsel-1.6.0-py2.py3-none-any.whl (13 kB)
  Using cached cssselect-1.1.0-py2.py3-none-any.whl (16 kB)
  Using cached queuelib-1.6.2-py2.py3-none-any.whl (13 kB)
  Using cached zope.interface-5.4.0-cp39-cp39-manylinux2010_x86_64.whl (255 kB)
  Using cached PyDispatcher-2.0.5-py3-none-any.whl
  Using cached service_identity-21.1.0-py2.py3-none-any.whl (12 kB)
  Using cached itemadapter-0.5.0-py3-none-any.whl (10 kB)
  Using cached tldextract-3.2.1-py3-none-any.whl (87 kB)
  Using cached itemloaders-1.0.4-py3-none-any.whl (11 kB)
  Using cached w3lib-1.22.0-py2.py3-none-any.whl (20 kB)
  Using cached Twisted-22.4.0-py3-none-any.whl (3.1 MB)
  Using cached lxml-4.8.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (6.9 MB)
  Usi

In [2]:
# Import libraries
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
from sqlalchemy import create_engine
import json
import pandas as pd
import glob

def formatting_for_url(string):
    return string.replace(" ", "+")

search_keywords = formatting_for_url(input("Please enter your search keywords : "))
search_location = formatting_for_url(input("Please enter the name of the city : "))

Please enter your search keywords :  restaurant japonais
Please enter the name of the city :  marseille


In [3]:
################### Extract ################### 
class YelpSpider(scrapy.Spider):

    # Name of your spider
    name = "yelp"

    # Url to start your spider from 
    start_urls = [
        'https://www.yelp.fr/search?find_desc={}&find_loc={}'.format(search_keywords,search_location),
    ]

    def parse(self, response):
        
        paragraphs = response.css('a.css-1m051bw')
        #page_number = response.css(' li > div > div > span.css-chan6m::text').getall()[0].split()[2]
        
        for p in paragraphs[3:]: 
            yield {
                 'search_keywords' : search_keywords,
                'search_location' : search_location,
                'company': p.css('::text').getall()
            }
        
        try:
            # Select the NEXT button and store it in next_page
            next_page = response.css('a.next-link').attrib["href"]
        except KeyError:
            # In the last page, there won't be any "href" and a KeyError will be raised
            logging.info('No next page. Terminating crawling process.')
        else:
            # If a next page is found, execute the parse method once again
            yield response.follow(next_page, callback=self.parse)
            
# Result file
filename = "{}-{}.json".format(search_keywords,search_location)

# Declare a new CrawlerProcess 
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'results/' + filename : {"format": "json"},
    }
})

# Start the crawling using the spider you defined above
process.crawl(YelpSpider)
process.start()


################### Transform json files to Dataframe ###################
init_df = pd.DataFrame()
for file in glob.glob("results/*.json"):
    df = pd.read_json(file)
    init_df = pd.concat([init_df,df])

    
################### Load to Posgresql Server ###################

HOSTNAME = "your_hostname"
USERNAME = "your_username"
PASSWORD = "your_password"
DBNAME = "your_dbname"
PORT = "5432"

engine = create_engine("postgresql+psycopg2://{}:{}@{}/{}".format(USERNAME,PASSWORD,HOSTNAME,DBNAME), echo=True)
# Save in a table named restaurant
init_df.to_sql("restaurant", engine)

2022-04-18 11:33:32 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-04-18 11:33:32 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:20:46) - [GCC 9.4.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 36.0.1, Platform Linux-5.4.170+-x86_64-with-glibc2.31
2022-04-18 11:33:32 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2022-04-18 11:33:32 [scrapy.extensions.telnet] INFO: Telnet Password: 1b6789140629c4e1
2022-04-18 11:33:32 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-04-18 11:33:32 [scrapy.middleware] INFO: Ena

2022-04-18 11:33:51,040 INFO sqlalchemy.engine.Engine select pg_catalog.version()


2022-04-18 11:33:51 [sqlalchemy.engine.Engine] INFO: select pg_catalog.version()


2022-04-18 11:33:51,042 INFO sqlalchemy.engine.Engine [raw sql] {}


2022-04-18 11:33:51 [sqlalchemy.engine.Engine] INFO: [raw sql] {}


...

2022-04-18 11:33:52,184 INFO sqlalchemy.engine.Engine [generated in 0.00392s] ({'index': 0, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Izumi']}, {'index': 1, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Moshi Moshi']}, {'index': 2, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Kisoro']}, {'index': 3, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Jomon Negishi']}, {'index': 4, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Akira']}, {'index': 5, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Sakura']}, {'index': 6, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Le Shuriken']}, {'index': 7, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Sushi Wafu']}  ... displaying 10 of 563 total bound parameter

2022-04-18 11:33:52 [sqlalchemy.engine.Engine] INFO: [generated in 0.00392s] ({'index': 0, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Izumi']}, {'index': 1, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Moshi Moshi']}, {'index': 2, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Kisoro']}, {'index': 3, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Jomon Negishi']}, {'index': 4, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Akira']}, {'index': 5, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Sakura']}, {'index': 6, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Le Shuriken']}, {'index': 7, 'search_keywords': 'restaurant+japonais', 'search_location': 'lille', 'company': ['Sushi Wafu']}  ... displaying 10 of 563 total bound parameter 

2022-04-18 11:33:52,453 INFO sqlalchemy.engine.Engine COMMIT


2022-04-18 11:33:52 [sqlalchemy.engine.Engine] INFO: COMMIT


563